# ASDF and World Coordinate Systems

The handling of World Coordinate Systems (WCS) is inextricably bound to 
digital observational data.t For example, given a pixel in the data array 
from a detector, what RA, Dec does it map to? Or what pixel does an 
RA, Dec map to? Likewise for spectrographic data, what wavelength, 
RA, and Dec does a pixel correspond to? Astronomers have had various 
solutions to this problem in the past. Most of them have been awkward
to deal with.

Dealing with WCS issues flexibly was a major driver for ASDF and it 
deserves its own tutorial.

## FITS and WCS

FITS has had a series of papers and resulting standards to deal with
WCS issues. Generally speaking, these have presumed mostly issues of 
idealized detectors and optics, where distortion is not a significant
factor. There was much focus on handling many kinds of map projecions
for images, but little provision for distortions beyond what projections
introduce. Likewise, most of the spectrosopic WCS models either presume
specific dispersion models, or the old fallback of wavelength tables,
which to this day many astronomers consider the only solution to 
spectroscopic WCS.

Handling distortions in FITS has met with a failure to settle on 
any general approach. Currently there are only two practical imaging
solutions, namely using the SIP convention, or a pixel translation
table. The first is not suitable for high spatial position precision
space telescopes 
such as HST, JWST or NGRST, the second is wasteful of disk space
while being inflexible to adjustments (more on that later). Since
most spactrographs use effectively the pixel translation approach,
they are also wasteful of space for IFUs and long slit data, while
also being inflexible to adjustments.

## A more flexible approach

Ideally, one would want to be able to construct coordinate transforms
out of simpler models to form a pipeline of transformations from 
one coordinate frame to another. The combination of the Python packages 
astropy modeling, GWCS (Generalized World Coordinate Systems) and asdf
provide a very flexible and powerful means of dealing with WCS.

This tutorial will not address how one develops a WCS model as that
is almost always a complex issue requiring very deep understanding
of the instrument involved and careful treatment of coordinates, and
usually is best left to experts (though being able to tweak the 
parameters of such WCS models is something than can and often should
be exposed to users).

This tutorial will focus more on using existing WCS models and seeing
how they can be modified to improve results.

## Using GWCS with ASDF

This initial imaging example will use the same wcs that was in the
JWST data file used in this tutorial: [Reading a JWST ASDF file]
(Reading_a_JWST_ASDF_file.ipynb) except only the WCS has been saved
to its own file (ASDF makes saving components of ASDF files to their
own files trivial).

In [ ]:
import asdf
from astropy.utils.data import download_file
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10, 10)
from matplotlib import pyplot as plt
file1_url_1 = 'https://data.science.stsci.edu/redirect/ASDF/asdf_tut_gwcs.asdf'

In [ ]:
filepath = download_file(file1_url_1)
# Wait until download completes (when 'In [*]' changes the  '* to a number')
# Ignore warnings

In [ ]:
af = asdf.open(filepath)

In [ ]:
# Get the GWCS for these data
wcs = af['wcs']

In [ ]:
# Illustrate use in converting pixel coordinate to sky
wcs(100,200)

In [ ]:
# But what coordinate system for the outputs? What units?
wcs.output_frame

In [ ]:
# The WCS may have intermediate frames defined as well
wcs.pipeline

From this we see that there are three frames defined, the detector frame
(e.g., pixel coordinates), the v2v3 frame, and the wolrd coordinate frame,
which is the output frame.

It is possible to access the corresponding coordinates for any intermediate frame.

In [ ]:
v2v3coords = wcs.transform('detector','v2v3', 100, 200)
v2v3coords

In [ ]:
# Units?
wcs.pipeline[1].frame.unit

In [ ]:
# Complete computation to world
wcs.transform('v2v3', 'world', *v2v3coords)

## Relevance to ASDF?

OK, GWCS has some extra features. But why is this part of an ASDF tutorial? Mainly to illustrate that ASDF preserves all the intrinsic information about the WCS and does not restrict transforms to a finite set of tools as FITS WCS does. We will illustrate this by examining this WCS in more detail. First note that ASDF allows an arbitrary number of intermediate frames (especially useful for spectral data), as well as an arbitrary number of different WCS models within a file. ASDF provides a way of saving most of astropy's models, of which there is a large number. More important, it can save astropy compound models. Arbitrarily complex combinations of models can be used and saved in ASDF, including all the projection models that FITS provides.

So while most of this tutorial is focused on GWCS, it is ASDF that enables this to be used with data files without forcing the user to obtain reference files and special software to determine the correct WCS as had to be done during most of HST lifetime.

Let's look at the details of the first step's transform.

In [ ]:
transform1 = wcs.pipeline[0].transform
type(transform1)

In [ ]:
print(transform1)

This compound model is sufficiently complicated to require some detailed explanation (but not nearly as complicated as some JWST spectral WCS models!). First a brief explanation of the notation. Inputs and outputs indicate that this model takes two input values (or two arrays of values and generates two output values, i.e., two input coordinates and two output coordinates. Expression is the expression used to combine astropy models into a more complex compound model. To keep the expression concise, each term is replaced by a number and the model associated with that number is listed below. The two operators in particular that need special explanation are & and |. The & operator means that two functions are joined by adding dimensionality. If a Shift function shifts the value of the input by the given value, the joining two Shift functions shift the first input by the first value, and the second input by the second value. It is a way of making two 1D functions a 2D function, albeit with each input coordinate treated independently of the other. The | operator feeds the output of the preceding function into the next. The number of outputs of the preceding must match the number of inputs of the following function. Finally, the Mapping function acts as a switchboard between inputs and output. For example, Mapping(0, 1, 0, 1) takes two inputs and generates 4 outputs, where outputs 0, and 2 are both equal to the first input, and outputs 1 and 3 are both equal to the second input.

Now to summarize in words what this transform does: shifts both input values first by 1 and then by -1024.5. This to make the coordinates relative to the center of the detector and cente of that pixel. The next step splits the two input values so that both go to two different 2D polynomials (each of which produces only one value, so the net result is two output value. Then these values go through a similar split and to a second polynomial that is effectively applying a small rotation, followed by a shift in the V2,v3 coordinate system.

Now to look at the second step transform.

In [ ]:
transform2 = wcs.pipeline[1].transform
print(transform2)

This transform is much simpler and consists of a scale change then conversion to cartesian to perform Euler rotations to the RA, Dec system and then conversion back to spherical coordinates.

Note that there is no use of a tangent projection, and such distortions that would result from that have been folded into the polynomial distortion model.

## Modifying the WCS

This is a somewhat contrived example, but it is to illustrate that one can make customizations to a WCS fairly easily. In this particular case one could change any of the parameters in the transforms, and save the to an ASDF file without any concern for how to do that. Or one can modify the existing transform by adding new elements to the transform. So supposing the existing plate scale used does not include velocity aberration, then one would want to include that as well as a plate scale change, and perhaps not change  the existing plate scale application to be clear there are two different scales applied. 



In [ ]:
from astropy.modeling.models import Scale
va_plate_scale_correction = 1.00008
#transform3 = (Scale(va_plate_scale_correction) & Scale(va_plate_scale_correction)) | transform2
wcs.pipeline[1].transform = transform3
af.write_to('modified_wcs.asdf', all_array_storage='inline')

In [ ]:
af2 = asdf.open('modified_wcs.asdf')

In [ ]:
wcs2 = af2['wcs']
mtransform2 = wcs2.pipeline[1].transform
print(mtransform2)

For those interested, here are the actual contents of the asdf file

In [ ]:
with open('modified_wcs.asdf') as asdftext: print(asdftext.read()) 

The result is fairly extensive and not nearly as easy to interpret as the python printed version. Nevertheless, it demonstrates ASDF is capable of saving arbitrarily complex expressions without the user needed to understand the details of saving reading those expressions.